# Steps to follow 
1. Data cleaning
2. EDA
3. Feature Engineering

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [44]:
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/playstore-Dataset/main/googleplaystore.csv')
df.head()
df_copy= df.copy()

In [38]:
df.shape


(10841, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [7]:
df.describe()

,Rating
count,9367.000000
mean,4.193338
std,0.537431
min,1.000000
25%,4.000000
50%,4.300000
75%,4.500000
max,19.000000


In [8]:
# We will start off with checking of null vlaues
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

### Insights and observation 
Dataset has missing vlaues

In [45]:
#Check if all the values in reviews are numeric
df['Reviews'].unique() 
# String format - now to find out if there are any numerical values
df['Reviews'].str.isnumeric().sum()
# #Check if all the values in reviews are numeric
df['Reviews'].unique() 
# String format - now to find out if there are any numerical values
df['Reviews'].str.isnumeric().sum()
# np.int64(10840) - 1 column is not in the numeric format
df[~df['Reviews'].str.isnumeric()]
# Here the reviews is 3M now we need to covnert this 3M into a numeric value 
def convert_reviews(val):
    if isinstance(val, str):
        val = val.strip()
        if val.endswith('M'):
            return float(val[:-1]) * 1_000_000
        elif val.endswith('K'):
            return float(val[:-1]) * 1_000
        elif val.isnumeric():
            return int(val)
    return None  # or np.nan if you prefer

df_copy['Reviews'] = df_copy['Reviews'].apply(convert_reviews)

In [ ]:
# Another way to convert the M and K to numbers is 
# df['Size'].str.replace('M', '000000') - but this only works if there is no decimal then 2.7M would become 2.7000000 which is 2.7 only
def convert_size(val):
    if isinstance(val,str):
        if val.endswith('M'):
            return float(val[:-1])*1000000
        elif val.endswith('k'):
            return float(val[:-1])*1000
        else:
            print(val)
            try:
                return float(val)
            except: # incase ther is a string like "Varies with device"
                return np.nan
    return None

df_copy['Size']= df_copy['Size'].apply(convert_size)

In [ ]:
df_copy['Installs'].unique() # This has + signs eg 2M+
df_copy['Installs'] =df_copy['Installs'].str.replace('+','')
df_copy['Installs'] = df_copy['Installs'].str.replace('Free','0')
df_copy['Installs'] = df_copy['Installs'].str.replace(',','')
df_copy['Installs']= df_copy['Installs'].astype('float')
df_copy['Installs'].unique() # This has + signs eg 2M+
df_copy.describe()

,Rating,Reviews,Size,Installs
count,9367.000000,1.084100e+04,9.145000e+03,1.084100e+04
mean,4.193338,4.443887e+05,2.151653e+07,1.546291e+07
std,0.537431,2.927728e+06,2.258875e+07,8.502557e+07
min,1.000000,0.000000e+00,8.500000e+03,0.000000e+00
25%,4.000000,3.800000e+01,4.900000e+06,1.000000e+03
50%,4.300000,2.094000e+03,1.300000e+07,1.000000e+05
75%,4.500000,5.479800e+04,3.000000e+07,5.000000e+06
max,19.000000,7.815831e+07,1.000000e+08,1.000000e+09


In [ ]:
df_copy['Price'].unique() # This has $ signs
df_copy['Price'] =df_copy['Price'].str.replace('$','')
df_copy['Price'] = df_copy['Price'].str.replace('Everyone','0')
df_copy['Price']= df_copy['Price'].astype('float')
df_copy['Price'].unique() 

In [68]:
df_copy['Last Updated'].unique()

array(['January 7, 2018', 'January 15, 2018', 'August 1, 2018', ...,
       'January 20, 2014', 'February 16, 2014', 'March 23, 2014'],
      shape=(1378,), dtype=object)

In [ ]:
df_copy['Last Updated'] = pd.to_datetime(df_copy['Last Updated'], errors='coerce')
df_copy = df_copy[df_copy['Last Updated'].notnull()]
df_copy['Day'] = df_copy['Last Updated'].dt.day
df_copy['Month'] = df_copy['Last Updated'].dt.month
df_copy['Year'] = df_copy['Last Updated'].dt.year


In [75]:
df_copy.to_csv('data/google.csv')